# Quantifying alternative splicing from RNA-seq data

This pipeline implements our pipeline to call alternative splicing events from RNA-seq data, using [`leafcutter`](https://www.nature.com/articles/s41588-017-0004-9) and [`psichomics`](https://academic.oup.com/nar/article/47/2/e7/5114259) to call the RNA-seq data from original `fastq.gz` data. It implements the GTEx pipeline for GTEx/TOPMed project. Please refer to [this page](https://github.com/broadinstitute/gtex-pipeline/blob/master/TOPMed_RNAseq_pipeline.md) for detail. The choice of pipeline modules in this project is supported by internal (unpublished) benchmarks from GTEx group.

**Various reference data needs to be prepared before using this workflow**. [Here we provide a module](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/reference_data.html) to download and prepare the reference data. 

The product of this workflow can be used in generating phenotype tables using /molecular_phenotyles/QC/splicing_normalization.ipynb.

## Methods overview

There are many types of alternative splicing events. See [Wang et al (2008)](https://pubmed.ncbi.nlm.nih.gov/18978772/) and [Park et al (2018)](https://pubmed.ncbi.nlm.nih.gov/29304370/) for an illustration on different events and how splicings are controlled. We will apply two methods to quantify alternative splicing:

1. [`psichomics`](https://academic.oup.com/nar/article/47/2/e7/5114259) that quantifies each specific event. In particular the exon skipping event which is used also in GTEx sQTL analysis.
2. [`leafcutter`](https://www.nature.com/articles/s41588-017-0004-9) to quantify the usage of alternatively excised introns. This collectively captures skipped exons, 5’ and 3’ alternative splice site usage and other complex events. The method was previously applied to ROSMAP data as part of the Brain xQTL version 2.0. 

## Input

### `leafcutter`

The bam file can be generated by `the STAR_align` workflow from our RNA_calling.ipynb module. 

A meta-data file, white space delimited without header, containing 3 columns: sample ID, RNA strandness and path to the BAM file:

```
sample_1 rf samp1.bam
sample_2 fr samp2.bam
sample_3 unstranded samp3.bam
```

All the BAM files should be available under specified folder (default assumes the same folder as where the meta-data file is).

If intend to blacklist some chromosomes and not analyze it, add one text file named black_list.txt with one chromosome name per line in the same directory of the meta-data file.


### `psichomics`

A meta-data file, white space delimited without header, containing 3 columns: sample ID, RNA strandness and path to the SJ.out.tab files:

```
sample_1 rf samp1SJ.out.tab
sample_2 fr samp2SJ.out.tab
sample_3 unstranded samp3SJ.out.tab
```

All the SJ.out.tab files should be available under specified folder (default assumes the same folder as where the meta-data file is).



## Output

### `leafcutter`

`{sample_list}` below refers to the name of the meta-data file input.

Main output include: 

- `{sample_list}_intron_usage_perind.counts.gz` file with row id in format: "chromosome:intron_start:intron_end:cluster_id", column labeled as input sample names and each type of intron usage ratio under each sample (i.e. #particular intron in a sample / #total introns classified in the same cluster in a sample) in each cells. 
- `{sample_list}_intron_usage_perind_numers.counts.gz` file with the same row and column label but the count of each intron in each cells.

### `psichomics`

- `psi_raw_data.tsv` A dataframe of PSI values (quantification of the alternative splicing events) with first column splicing event identifier (for instance, SE_1_-_2125078_2124414_2124284_2121220_C1orf86) is composed of:

                   Event type (SE stands for skipped exon)
                   Chromosome (1)
                   Strand (-)
                   Relevant coordinates depending on event type (in this case, the first constitutive exon’s end, the                            alternative exon’ start and end and the second constitutive exon’s start)
                   Associated gene (C1orf86)

| Splicing Event Type | Abbreviation | [Coordinates](https://bioconductor.org/packages/release/bioc/manuals/psichomics/man/psichomics.pdf) |
| --- | --- | --- |
| Skipped Exon | SE | constitutive exon 1 end, alternative exon (start and end) and constitutive exon 2 start |
| Mutually exclusive exon | MXE | constitutive exon 1 end, alternative exon 1 and 2 (start and end) and constitutive exon 2 start |
| Alternative 5' splice site | A5SS | constitutive exon 1 end, alternative exon 1 end and constitutive exon 2 start |
| Alternative 3' splice site | A3SS | constitutive exon 1 end, alternative exon 1 start and constitutive exon 2 start |
| Alternative first exon | AFE | constitutive exon 1 end, alternative exon 1 end and constitutive exon 2 start |
| Alternative last exon | ALE | constitutive exon 1 end, alternative exon 1 start and constitutive exon 2 start |
| Alternative first exon (exon-centered - less reliable) | AFE_exon | constitutive exon 1 end, alternative exon 1 end and constitutive exon 2 start |
| Alternative last exon (exon-centered - less reliable) | ALE_exon | constitutive exon 1 end, alternative exon 1 start and constitutive exon 2 start |



## Minimal working example


### For `leafcutter`
A minimal working example is uploaded in the [google drive](https://drive.google.com/drive/folders/1lpcx3eKG2UpauntLUuJ6bMBjHyIhWW_R?usp=sharing)

In [ ]:
sos run pipeline/splicing_calling.ipynb leafcutter \
    --cwd output/ \
    --samples sample_bam.list \
    --container containers/leafcutter.sif 

### For `psichomics`

In [ ]:
sos run splicing_calling.ipynb psichomics \
    --cwd psidata/output/ \
    --samples psidata/sample_SJ.list \
    --container container/psichomics.sif

## Command interface

In [40]:
sos run splicing_calling.ipynb -h

usage: sos run splicing_calling.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  leafcutter
  psichomics

Global Workflow Options:
  --cwd output (as path)
                        The output directory for generated files.
  --samples VAL (as path, required)
                        Sample meta data list
  --data-dir  path(f"{samples:d}")

                        Raw data directory, default to the same directory as
                        sample list
  --job-size 1 (as int)
                        For cluster jobs, number commands to run per job
  --walltime 5h
                        Wall clock time expected
  --mem 16G
                        Memory expected
  --nu

## Setup and global parameters

In [17]:
[global]
# The output directory for generated files. 
parameter: cwd = path("output")
# Sample meta data list
parameter: samples = path
# Raw data directory, default to the same directory as sample list
parameter: data_dir = path(f"{samples:d}")
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 8
# Software container option
parameter: container = ""
from sos.utils import expand_size
cwd = path(f'{cwd:a}')

def get_samples(fn, dr):
    import os
    samples = [x.strip().split() for x in open(fn).readlines()]
    names = []
    strandness = []
    files = []
    datas = []
    
    for i, x in enumerate(samples):
        if len(x)<3:
            raise ValueError(f"Line {i+1} of file {fn} must have 3 columns")
        names.append(x[0])
        strandness.append(x[1])
        files.append(x[2])
        
    for j in range(len(strandness)):
        # for regtools command usage, replace 0 = unstranded/XS, 1 = first-strand/RF, 2 = second-strand/FR
        if strandness[i] == 'rf':
            strandness[i] = 1
        if strandness[i] == 'fr':
            strandness[i] = 2
        if strandness[i] == 'unstranded':
            strandness[i] = 0
            
    for y in files:
        y = os.path.join(dr, y)
        if not os.path.isfile(y):
            raise ValueError(f"File {y} does not exist")
        datas.append(y)
        
    if len(files) != len(set(files)):
        raise ValueError("Duplicated files are found (but should not be allowed) in BAM file list")
        
    return names, datas, strandness

sample_id, data, strandness = get_samples(samples, data_dir)

## `leafcutter`

Documentation: [`leafcutter`](https://davidaknowles.github.io/leafcutter/index.html). The choices of regtool parameters are [discussed here](https://github.com/davidaknowles/leafcutter/issues/127).

### Other clustering options:

*   "-q", "--quiet" : don't print status messages to stdout, default=True.

*   "-p", "--mincluratio" : minimum fraction of reads in a cluster that support a junction, default 0.001. 

*   "-c", "--cluster" : refined cluster file when clusters are already made, default = None.

*   "-k", "--nochromcheck" : Don't check that the chromosomes are well formated e.g. chr1, chr2, ..., or 1, 2, ..., default = False.

*    "-C", "--includeconst" : also include constitutive introns, default = False.

The default parameter we used are:

`--min_clu_ratio 0.001 --max_intron_len 500000 --min_clu_reads 30`

These parameter is based on [GTEX's sQTL discovery pipeline (Section 3.4.3) ](https://www.science.org/action/downloadSupplement?doi=10.1126%2Fscience.aaz1776&file=aaz1776_aguet_sm.pdf)

### Things to keep in mind:

* If .bam.bai index files of the .bam input are ready before using leafCutter, it can be placed in the same directory with input .bam files and the "samtools index ${_input}" line can be skipped.


In [18]:
[leafcutter_1]
# anchor length (default 8)
parameter: anchor_len = 8
# minimum intron length to be analyzed (default 50)
parameter: min_intron_len = 50
# maximum intron length to be analyzed (default 500000)
parameter: max_intron_len = 500000
input: data, group_by = 1, group_with = "strandness"
output: f'{cwd}/{_input:bn}.junc' 
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    samtools index ${_input}
    regtools junctions extract -a ${anchor_len} -m ${min_intron_len} -M ${max_intron_len} -s ${_strandness} ${_input} -o ${_output}

In [19]:
[leafcutter_2]
# minimum reads in a cluster (default 50 reads)
parameter: min_clu_reads = 30 
# maximum intron length to be analyzed (default 500000)
parameter: max_intron_len = 500000 
# minimum fraction of reads in a cluster that support a junction (default 0.001)
parameter: min_clu_ratio = 0.001
input: group_by = 'all'
output: f'{cwd}/{samples:bn}_intron_usage_perind.counts.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container=container
    rm -f ${_output:nn}.junc
    for i in ${_input:r}; do
    echo $i >> ${_output:nn}.junc ; done
    python /opt/leafcutter/clustering/leafcutter_cluster_regtools.py -j ${_output:nn}.junc -o ${f'{_output:bnn}'.replace("_perind","")} -m ${min_clu_reads} -l ${max_intron_len} -r ${cwd} -p ${min_clu_ratio}

## `psichomics`

Documentation: [`psichomics`](http://bioconductor.org/packages/release/bioc/html/psichomics.html)

### Other options

quantifySplicing( annotation,
                  junctionQuant,
                  eventType = c("SE", "MXE", "ALE", "AFE", "A3SS", "A5SS"),
                  minReads = 10,
                  genes = NULL
)

In function quantifySplicing, arguments eventType (Character: splicing event types to quantify), minReads (Integer: values whose number of total supporting read counts is below minReads are returned as NA) and genes (Character: gene symbols for which to quantify splicing events. If NULL, events from all genes are quantified.) can be specified. Usage and default values are shown above.

### Things to keep in mind
 
* The default annotation used in psichomics Human hg38 (2018-04-30), provided by psichomics package. Descrption of the database can be found in the Alternative splicing annotation section in the [MATERIALS AND METHODS](https://academic.oup.com/nar/article/47/2/e7/5114259?login=true#130023625) part. 
* The current .out.tab outputs in upstream workflow is in .gz format, if that is kept some more zcat command might be added at the begining
* Not all events in the "AH63657" have corresponding ENSG gene ID, so currently using gene names at the end of event IDs, to be changed later.

In [ ]:
[psichomics_1]
parameter: splicing_annotation = "AH63657"
input: data
output: f'{cwd}/psi_raw_data.tsv'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    library("psichomics")
    library("purrr")
    library("dplyr")
    library("tidyr")
    setwd("${samples:d}/output/")
    prepareJunctionQuant(${_input:r,})
    data <- loadLocalFiles("${cwd}")
    junctionQuant <- data[[1]]$`Junction quantification`
    # if user provide any annotation, use that rds file, or usr the default 
    if (substr("${splicing_annotation}", nchar("${splicing_annotation}")-3, nchar("${splicing_annotation}")) == ".rds"){
    annotation = readRDS("${splicing_annotation}")
    } else {
    annotation <- loadAnnotation("${splicing_annotation}")
    #annotation <- map(annotation, ~.x%>%separate(`VAST-TOOLS.Event.ID`, sep = "_",into = "Gene")%>%separate(`Gene`, sep = "-",into = "Gene"))
    }
    psi <- quantifySplicing(annotation, junctionQuant)
    write.table(psi, file='${cwd}/psi_raw_data.tsv', quote=FALSE, sep='\t')